In [1]:
import graphlab
import pandas as pd
import numpy as np

In [2]:
temp=pd.read_csv(r"C:\Users\yvonn\Downloads\result.csv",sep='\t',encoding="utf-8",)

In [3]:
temp.columns

Index([u'user_id_x', u'individual_x', u'team_x', u'gender_x',
       u'constellation_x', u'user_charact_x', u'region_x', u'rank_x',
       u'user_age_x', u'user_work_id', u'activity', u'score', u'activitytime',
       u'user_work_type', u'user_work_mod', u'user_work_scene', u'user_id_y',
       u'song_id', u'time', u'individual_y', u'team_y', u'gender_y',
       u'constellation_y', u'user_charact_y', u'region_y', u'rank_y',
       u'user_age_y', u'song_theme', u'song_entert', u'song_age', u'song_lang',
       u'song_genre', u'song_location', u'song_key', u'song_opera',
       u'song_trend', u'song_title', u'song_spec', u'song_type',
       u'song_video_rank', u'singer_id', u'song_video', u'song_audio',
       u'singer_m', u'singer_f', u'singer_g', u'singer_sex', u'singer_name',
       u'singer_age_range', u'singer_location'],
      dtype='object')

## Split train and test

In [5]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(temp,
                                   stratify=temp['user_id_x'], 
                                   test_size=0.20,
                                   random_state=42)
ratings_train=train[['user_work_id','user_id_x','score']]
ratings_test=test[['user_work_id','user_id_x','score']]
train_data_matrix=ratings_train.groupby(['user_id_x','user_work_id']).sum().reset_index().pivot_table(values='score',index='user_work_id',columns='user_id_x',aggfunc=np.sum).replace(np.nan,0).T
test_data_matrix=ratings_test.groupby(['user_id_x','user_work_id']).sum().reset_index().pivot_table(values='score',index='user_work_id',columns='user_id_x',aggfunc=np.sum).replace(np.nan,0).T

## similarity between items and users

In [7]:
from sklearn.metrics.pairwise import pairwise_distances
user_similarity = pairwise_distances(train_data_matrix, metric='cosine')
item_similarity = pairwise_distances(train_data_matrix.T, metric='cosine')

## Make prediction

In [8]:
def predict(ratings, similarity, type='user'):
    if type == 'user':
        mean_user_rating = ratings.mean(axis=1)
        #You use np.newaxis so that mean_user_rating has same format as ratings
        ratings_diff = ratings.sub(mean_user_rating,axis=0)
        pred1 = np.array(mean_user_rating) + similarity.dot(ratings_diff).T / np.abs(similarity).sum(axis=1)
        pred=pd.DataFrame(pred1.T)
    elif type == 'item':
        pred = ratings.dot(similarity) / np.abs(similarity).sum(axis=1)
    return pred

In [242]:
#train_data_matrix.dot(item_similarity)/np.abs(item_similarity).sum(axis=1)

In [9]:
item_prediction = predict(train_data_matrix, item_similarity, type='item')
user_prediction = predict(train_data_matrix, user_similarity, type='user')

In [135]:
prediction_item=pd.DataFrame({n: item_prediction.T[col].nlargest(10).index.tolist() 
                  for n, col in enumerate(item_prediction.T)}).T
result_item_1=pd.DataFrame()
for i in range(100):
    rec=temp[temp['user_work_id'].isin(prediction_item.iloc[i])][['user_charact_y','song_lang','song_type']].drop_duplicates().apply(tuple, axis=1).reset_index().drop(['index'],axis=1).T
    result_item_1=result_item_1.append(rec)
result_item_1

,0,1,2,3,4,5,6,7,8,9
0,"(萝莉, 国语, 电子)","(正太, 国语, 新世纪)","(硬汉, 韩语, 动漫)","(萝莉, 粤语, 新世纪)","(硬汉, 英语, 新世纪)","(硬汉, 小语种, 布鲁斯)","(鲜肉, 国语, 布鲁斯)","(正太, 泰语, 流行)","(女神, 小语种, 新世纪)","(正太, 粤语, 亲子)"
0,"(硬汉, 粤语, 传统)","(女神, 小语种, 传统)","(正太, 韩语, 嘻哈)","(硬汉, 英语, 雷鬼)","(御姐, 闽南语, 舞曲)","(帅哥, 泰语, 爵士)","(美女, 小语种, 传统)","(正太, 日语, 重金属)","(大叔, 韩语, 流行)","(美女, 韩语, 雷鬼)"
0,"(御姐, 泰语, 重金属)","(硬汉, 小语种, 亲子)","(鲜肉, 国语, 摇滚)","(鲜肉, 小语种, 布鲁斯)","(御姐, 泰语, 舞曲)","(御姐, 韩语, 嘻哈)","(女神, 国语, 亲子)","(美女, 韩语, 布鲁斯)","(硬汉, 日语, 重金属)","(鲜肉, 小语种, 经典)"
0,"(御姐, 小语种, 经典)","(硬汉, 英语, 乡村)","(鲜肉, 英语, 爵士)","(大叔, 英语, 乡村)","(御姐, 国语, 动漫)","(大叔, 英语, 流行)","(大叔, 韩语, 流行)","(正太, 英语, 动漫)","(美女, 国语, 经典)","(正太, 小语种, 摇滚)"
0,"(硬汉, 闽南语, 重金属)","(女神, 日语, 亲子)","(女神, 闽南语, 重金属)","(御姐, 韩语, 嘻哈)","(美女, 韩语, 嘻哈)","(女神, 小语种, 布鲁斯)","(硬汉, 英语, 新世纪)","(御姐, 闽南语, 雷鬼)","(女神, 韩语, 动漫)","(硬汉, 韩语, 新世纪)"
0,"(美女, 闽南语, 布鲁斯)","(大叔, 国语, 嘻哈)","(硬汉, 韩语, 乡村)","(正太, 国语, 流行)","(帅哥, 闽南语, 摇滚)","(美女, 小语种, 嘻哈)","(美女, 日语, 爵士)","(女神, 国语, 电子)","(大叔, 小语种, 爵士)","(美女, 泰语, R&B)"
0,"(大叔, 闽南语, 传统)","(硬汉, 韩语, 亲子)","(硬汉, 英语, 嘻哈)","(美女, 闽南语, 雷鬼)","(萝莉, 泰语, 爵士)","(硬汉, 泰语, 亲子)","(美女, 英语, 电子)","(女神, 日语, 舞曲)","(硬汉, 韩语, 经典)","(萝莉, 粤语, 嘻哈)"
0,"(硬汉, 闽南语, R&B)","(美女, 日语, R&B)","(硬汉, 国语, 摇滚)","(正太, 韩语, 亲子)","(萝莉, 英语, 重金属)","(美女, 国语, 电子)","(正太, 闽南语, 摇滚)","(御姐, 国语, 经典)","(萝莉, 闽南语, 流行)","(帅哥, 粤语, 动漫)"
0,"(硬汉, 闽南语, 重金属)","(鲜肉, 国语, 新世纪)","(帅哥, 粤语, 新世纪)","(正太, 闽南语, 电子)","(御姐, 闽南语, 爵士)","(萝莉, 闽南语, 布鲁斯)","(御姐, 日语, 舞曲)","(鲜肉, 泰语, 嘻哈)","(萝莉, 英语, 传统)","(硬汉, 泰语, 爵士)"
0,"(女神, 韩语, 爵士)","(正太, 英语, 流行)","(正太, 闽南语, 舞曲)","(正太, 小语种, 流行)","(御姐, 泰语, 经典)","(美女, 闽南语, 雷鬼)","(正太, 英语, 亲子)","(御姐, 英语, 动漫)","(鲜肉, 小语种, 传统)","(鲜肉, 小语种, 亲子)"


In [206]:
prediction_user=pd.DataFrame({n: user_prediction.T[col].nlargest(10).index.tolist() 
                  for n, col in enumerate(user_prediction.T)}).T
result_user_1=pd.DataFrame()
for i in range(100):
    rec=temp[temp['user_work_id'].isin(prediction_user.iloc[i])][['user_charact_y','song_lang','song_type']].drop_duplicates().apply(tuple, axis=1).reset_index().drop(['index'],axis=1).T
    result_user_1=result_user_1.append(rec)
    
result_user_1

,0,1,2,3,4,5,6,7,8,9
0,"(女神, 闽南语, 经典)","(硬汉, 日语, 乡村)","(萝莉, 泰语, 爵士)","(美女, 国语, 摇滚)","(御姐, 韩语, R&B)","(鲜肉, 国语, 摇滚)","(美女, 粤语, R&B)","(大叔, 韩语, 流行)","(鲜肉, 小语种, R&B)","(正太, 小语种, 摇滚)"
0,"(女神, 闽南语, 经典)","(硬汉, 日语, 乡村)","(御姐, 韩语, 经典)","(萝莉, 泰语, 爵士)","(美女, 国语, 摇滚)","(鲜肉, 国语, 摇滚)","(美女, 粤语, R&B)","(大叔, 韩语, 流行)","(鲜肉, 小语种, R&B)","(正太, 小语种, 摇滚)"
0,"(女神, 闽南语, 经典)","(硬汉, 日语, 乡村)","(御姐, 韩语, 经典)","(萝莉, 泰语, 爵士)","(美女, 国语, 摇滚)","(御姐, 韩语, R&B)","(美女, 粤语, R&B)","(大叔, 韩语, 流行)","(鲜肉, 小语种, R&B)","(正太, 小语种, 摇滚)"
0,"(女神, 闽南语, 经典)","(硬汉, 日语, 乡村)","(御姐, 韩语, 经典)","(萝莉, 泰语, 爵士)","(美女, 国语, 摇滚)","(御姐, 韩语, R&B)","(美女, 粤语, R&B)","(大叔, 韩语, 流行)","(鲜肉, 小语种, R&B)","(正太, 小语种, 摇滚)"
0,"(女神, 闽南语, 经典)","(硬汉, 日语, 乡村)","(萝莉, 泰语, 爵士)","(美女, 国语, 摇滚)","(御姐, 韩语, R&B)","(鲜肉, 国语, 摇滚)","(美女, 粤语, R&B)","(大叔, 韩语, 流行)","(鲜肉, 小语种, R&B)","(正太, 小语种, 摇滚)"
0,"(女神, 闽南语, 经典)","(硬汉, 日语, 乡村)","(御姐, 韩语, 经典)","(萝莉, 泰语, 爵士)","(美女, 国语, 摇滚)","(御姐, 韩语, R&B)","(美女, 粤语, R&B)","(大叔, 韩语, 流行)","(鲜肉, 小语种, R&B)","(正太, 小语种, 摇滚)"
0,"(女神, 闽南语, 经典)","(硬汉, 日语, 乡村)","(御姐, 韩语, 经典)","(萝莉, 泰语, 爵士)","(美女, 国语, 摇滚)","(御姐, 韩语, R&B)","(美女, 粤语, R&B)","(大叔, 韩语, 流行)","(鲜肉, 小语种, R&B)","(正太, 小语种, 摇滚)"
0,"(女神, 闽南语, 经典)","(硬汉, 日语, 乡村)","(萝莉, 泰语, 爵士)","(美女, 国语, 摇滚)","(御姐, 韩语, R&B)","(鲜肉, 国语, 摇滚)","(美女, 粤语, R&B)","(大叔, 韩语, 流行)","(鲜肉, 小语种, R&B)","(正太, 小语种, 摇滚)"
0,"(女神, 闽南语, 经典)","(硬汉, 日语, 乡村)","(御姐, 韩语, 经典)","(萝莉, 泰语, 爵士)","(御姐, 韩语, R&B)","(鲜肉, 国语, 摇滚)","(美女, 粤语, R&B)","(大叔, 韩语, 流行)","(鲜肉, 小语种, R&B)","(正太, 小语种, 摇滚)"
0,"(女神, 闽南语, 经典)","(硬汉, 日语, 乡村)","(御姐, 韩语, 经典)","(萝莉, 泰语, 爵士)","(美女, 国语, 摇滚)","(美女, 粤语, R&B)","(大叔, 韩语, 流行)","(正太, 韩语, R&B)","(鲜肉, 小语种, R&B)","(正太, 小语种, 摇滚)"


# Test Accuracy
## first method

In [ ]:
testact=test.groupby('user_id_x').apply(lambda x:x[['user_charact_y','song_lang','song_type']].apply(tuple,axis=1)).reset_index()
num_item=[testact[testact['user_id_x']==i][0].isin(result_item_1.iloc[i]).sum() for i in range(100)]
num_user=[testact[testact['user_id_x']==i][0].isin(result_user_1.iloc[i]).sum() for i in range(100)]

In [227]:
print sum(num_item),sum(num_user)

49 65


## second method

In [197]:
testact2=test.groupby('user_id_x').apply(lambda x:x[['user_id_x','user_charact_y','song_lang','song_type']])
num_user2=[]
for i in range(100):
    rec=temp[temp['user_work_id'].isin(prediction_user.iloc[i])][['user_charact_y','song_lang','song_type']].drop_duplicates().mode(axis=0)
    num_user2.append(testact2[testact2['user_id_x']==i][['user_charact_y','song_lang','song_type']].isin(rec.to_dict('list')).sum(axis=1).sum())
num_item2=[]
for i in range(100):
    rec=temp[temp['user_work_id'].isin(prediction_item.iloc[i])][['user_charact_y','song_lang','song_type']].drop_duplicates().mode(axis=0)
    num_item2.append(testact2[testact2['user_id_x']==i][['user_charact_y','song_lang','song_type']].isin(rec.to_dict('list')).sum(axis=1).sum())

In [228]:
print sum(num_item2),sum(num_user2)

1584 1490


## baseline

In [219]:
def base():
    base=pd.DataFrame(np.random.randint(0,1000,size=(100, 10)))
    result_base_1=pd.DataFrame()
    for i in range(100):
        rec=temp[temp['user_work_id'].isin(base.iloc[i])][['user_charact_y','song_lang','song_type']].drop_duplicates().apply(tuple, axis=1).reset_index().drop(['index'],axis=1).T
        result_base_1=result_base_1.append(rec)

    basetestact=test.groupby('user_id_x').apply(lambda x:x[['user_charact_y','song_lang','song_type']].apply(tuple,axis=1)).reset_index()
    base_1=[basetestact[basetestact['user_id_x']==i][0].isin(result_base_1.iloc[i]).sum() for i in range(100)]
    basetestact2=test.groupby('user_id_x').apply(lambda x:x[['user_id_x','user_charact_y','song_lang','song_type']])
    base_2=[]
    for i in range(100):
        rec=temp[temp['user_work_id'].isin(base.iloc[i])][['user_charact_y','song_lang','song_type']].drop_duplicates().mode(axis=0)
        base_2.append(basetestact2[basetestact2['user_id_x']==i][['user_charact_y','song_lang','song_type']].isin(rec.to_dict('list')).sum(axis=1).sum())
    return [sum(base_1),sum(base_2)]

In [232]:
base_result=pd.DataFrame([base() for i in range(100)])

In [233]:
#base_result=base_result.mean(axis=0)
base_result.mean(axis=0)

0      56.96
1    1605.66
dtype: float64

In [205]:
from sklearn.metrics import mean_squared_error
from math import sqrt
def error(test):
    zero_data = np.zeros(shape=(100,1000))
    d = pd.DataFrame(zero_data, columns=train_data_matrix.columns)
    data=(d+test).replace(np.nan,0)
    item_prediction = predict(data, item_similarity, type='item')
    user_prediction = predict(data, user_similarity, type='user')
    a=np.sum(abs(item_prediction-data).sum())/100000
    b=np.sum(abs(user_prediction-data).sum())/100000
    return [a,b]
print 'User-based CF RMSE: ' + str(error(test_data_matrix)[1])
print 'Item-based CF RMSE: ' + str(error(test_data_matrix)[0])

User-based CF RMSE: 0.6529686265098995
Item-based CF RMSE: 0.6505830957332733


## user-user method2

In [247]:
temp1=train[['user_id_x','user_charact_y','score']]
scoretable=temp1.groupby(['user_id_x','user_charact_y']).sum().reset_index()
#temp1.groupby(['user_id_x','user_charact_x']).count()
traindata1=scoretable.pivot_table(values='score',index='user_id_x',columns='user_charact_y',aggfunc=np.sum).replace(np.nan,0)
temp2=train[['user_id_x','song_lang','score']]
scoretable2=temp2.groupby(['user_id_x','song_lang']).sum().reset_index()
temp3=train[['user_id_x','song_type','score']]
scoretable3=temp3.groupby(['user_id_x','song_type']).sum().reset_index()
traindata2=scoretable2.pivot_table(values='score',index='user_id_x',columns='song_lang',aggfunc=np.sum).replace(np.nan,0)
traindata3=scoretable3.pivot_table(values='score',index='user_id_x',columns='song_type',aggfunc=np.sum).replace(np.nan,0)
ff=pd.concat([traindata1,traindata2,traindata3],axis=1)


In [248]:
from sklearn.metrics.pairwise import pairwise_distances
user_similarity_2 = pairwise_distances(ff, metric='cosine')
#item_similarity = pairwise_distances(train_data_matrix.T, metric='cosine')

In [245]:
user_similarity_2

array([[0.        , 0.09378275, 0.10056876, ..., 0.06855308, 0.06124847,
        0.08282521],
       [0.09378275, 0.        , 0.05905815, ..., 0.08282684, 0.08111283,
        0.0758082 ],
       [0.10056876, 0.05905815, 0.        , ..., 0.0765911 , 0.07758093,
        0.07990186],
       ...,
       [0.06855308, 0.08282684, 0.0765911 , ..., 0.        , 0.0626053 ,
        0.0702388 ],
       [0.06124847, 0.08111283, 0.07758093, ..., 0.0626053 , 0.        ,
        0.11398632],
       [0.08282521, 0.0758082 , 0.07990186, ..., 0.0702388 , 0.11398632,
        0.        ]])

In [250]:
user_prediction_2 = predict(train_data_matrix, user_similarity_2, type='user')

In [255]:
prediction_user_2=pd.DataFrame({n: user_prediction_2.T[col].nlargest(10).index.tolist() 
                  for n, col in enumerate(item_prediction.T)}).T
user2=pd.DataFrame()
for i in range(100):
    rec=temp[temp['user_work_id'].isin(prediction_user_2.iloc[i])][['user_charact_y','song_lang','song_type']].drop_duplicates().apply(tuple, axis=1).reset_index().drop(['index'],axis=1).T
    user2=user2.append(rec)


In [256]:
testact=test.groupby('user_id_x').apply(lambda x:x[['user_charact_y','song_lang','song_type']].apply(tuple,axis=1)).reset_index()

num_user_method2=[testact[testact['user_id_x']==i][0].isin(user2.iloc[i]).sum() for i in range(100)]

In [257]:
testact2=test.groupby('user_id_x').apply(lambda x:x[['user_id_x','user_charact_y','song_lang','song_type']])
num_user2_method2=[]
for i in range(100):
    rec=temp[temp['user_work_id'].isin(prediction_user_2.iloc[i])][['user_charact_y','song_lang','song_type']].drop_duplicates().mode(axis=0)
    num_user2_method2.append(testact2[testact2['user_id_x']==i][['user_charact_y','song_lang','song_type']].isin(rec.to_dict('list')).sum(axis=1).sum())


In [258]:
print sum(num_user_method2),sum(num_user2_method2)

71 1691


## Conclusion
# based on random data method 2 is slightly better

In [8]:
graphlab.get_dependencies()


By running this function, you agree to the following licenses.

* libstdc++: https://gcc.gnu.org/onlinedocs/libstdc++/manual/license.html
* xz: http://git.tukaani.org/?p=xz.git;a=blob;f=COPYING
    
Extracting xz.
Extracting gcc-libs.
Copying gcc-libs into the installation directory.


In [5]:
train_data = graphlab.SFrame(ratings_base)
test_data = graphlab.SFrame(ratings_test)

This non-commercial license of GraphLab Create for academic use is assigned to xcui2@wpi.edu and will expire on February 26, 2019.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: C:\Users\yvonn\AppData\Local\Temp\graphlab_server_1519706532.log.0


In [12]:

#a=result.pivot_table(values='score',index='user_work_id',columns='user_id_x',aggfunc=np.sum)

In [6]:
item_sim_model = graphlab.item_similarity_recommender.create(train_data, user_id='user_id_x', item_id='user_work_id', target='score', similarity_type='pearson')

Recsys training: model = item_similarity

Preparing data set.

Data has 11200 observations with 100 users and 1000 items.

Data prepared in: 0.015616s

Training model from provided data.

...

| 120.742ms                           | 100              | 1000            |

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 1.12674s

In [8]:
item_sim_recomm = item_sim_model.recommend(users=range(0,6),k=5)
item_sim_recomm.print_rows(num_rows=25)

+-----------+--------------+---------------+------+
| user_id_x | user_work_id |     score     | rank |
+-----------+--------------+---------------+------+
|     0     |     431      | 19.9949526718 |  1   |
|     0     |     257      | 19.7276083639 |  2   |
|     0     |     867      | 19.6978633939 |  3   |
|     0     |     730      | 19.2113402486 |  4   |
|     0     |     425      | 18.7971974235 |  5   |
|     1     |     431      | 19.9886066824 |  1   |
|     1     |     257      | 19.7961295145 |  2   |
|     1     |     867      |  19.73788582  |  3   |
|     1     |     730      | 19.2523342074 |  4   |
|     1     |     425      | 18.8090749802 |  5   |
|     2     |     431      | 19.9692585372 |  1   |
|     2     |     257      | 19.7484995518 |  2   |
|     2     |     867      | 19.6684463308 |  3   |
|     2     |     730      | 19.1356162548 |  4   |
|     2     |     425      | 18.8258191024 |  5   |
|     3     |     431      | 19.9970905565 |  1   |
|     3     

In [10]:
model_performance = graphlab.compare(test_data,[item_sim_model])
graphlab.show_comparison(model_performance,[item_sim_model])

PROGRESS: Evaluate model M0

Precision and recall summary statistics by cutoff
+--------+-----------------+------------------+
| cutoff |  mean_precision |   mean_recall    |
+--------+-----------------+------------------+
|   1    |       0.03      | 0.00101822825961 |
|   2    |       0.03      | 0.00204203778342 |
|   3    |       0.03      | 0.00311712935851 |
|   4    |      0.025      | 0.00351712935851 |
|   5    |      0.028      | 0.00493338551614 |
|   6    | 0.0283333333333 | 0.00610261628537 |
|   7    |       0.03      | 0.00755458672872 |
|   8    |       0.03      | 0.00867322383874 |
|   9    | 0.0322222222222 | 0.0104768100375  |
|   10   |      0.031      | 0.0111341376237  |
+--------+-----------------+------------------+
[10 rows x 3 columns]

Model compare metric: precision_recall
Canvas is accessible via web browser at the URL: http://localhost:56955/index.html
Opening Canvas in default web browser.
